<a href="https://colab.research.google.com/github/mbahadirk/OBSS-Competition/blob/main/OBSS_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

import os
import zipfile

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c obss-intern-competition-2025


Saving kaggle.json to kaggle.json
 99% 987M/995M [00:01<00:00, 706MB/s]
100% 995M/995M [00:01<00:00, 933MB/s]


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import os
from datetime import datetime

save_dir = "/content/drive/MyDrive/blip_checkpoints"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
!pip install -q transformers torchvision sentencepiece datasets evaluate

In [3]:
import os
import pandas as pd
import torch
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import BlipProcessor, BlipForConditionalGeneration
import pickle

In [4]:
with zipfile.ZipFile("obss-intern-competition-2025.zip", "r") as zip_ref:
    zip_ref.extractall("obss_caption_data")

In [5]:
df = pd.read_csv("/content/obss_caption_data/train.csv")
print(f"Toplam veri: {len(df)}")

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

Toplam veri: 21367


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
torch.manual_seed(42)

In [7]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-vqa-base").to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [8]:
class CaptionDataset(Dataset):
    def __init__(self, dataframe, processor, image_dir):
        self.dataframe = dataframe.reset_index(drop=True)
        self.processor = processor
        self.image_dir = image_dir

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.image_dir, f"{row['image_id']}.jpg")
        image = Image.open(image_path).convert("RGB")
        caption = row['caption']

        inputs = self.processor(images=image, text=caption, padding="max_length", truncation=True, return_tensors="pt")
        inputs["labels"] = inputs.input_ids.clone()
        for k in inputs:
            inputs[k] = inputs[k].squeeze()
        return inputs


In [16]:
train_dataset = CaptionDataset(train_df, processor, "obss_caption_data/train/train")
val_dataset = CaptionDataset(val_df, processor, "obss_caption_data/train/train")

train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=6, shuffle=False, pin_memory=True)

In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scaler = torch.cuda.amp.GradScaler()

<ipython-input-17-d1ea9cb8f0ff>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [18]:
num_epochs = 10
patience = 3
min_val_loss = float('inf')
early_stopping = 0
tracking_info = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        input_ids = batch['input_ids'].to(device)
        pixel_values = batch['pixel_values'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids = batch['input_ids'].to(device)
            pixel_values = batch['pixel_values'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            with torch.cuda.amp.autocast():
                outputs = model(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
            val_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    tracking_info.append((avg_train_loss, avg_val_loss, optimizer.param_groups[0]['lr']))

    print(f"\nEpoch {epoch+1}: Train Loss = {avg_train_loss:.4f} | Val Loss = {avg_val_loss:.4f}")

    model_filename = f"epoch_{epoch+1:02d}_val{avg_val_loss:.4f}.pt"
    model_path = os.path.join(save_dir, model_filename)
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to: {model_path}")

    if avg_val_loss < min_val_loss:
        torch.save(model.state_dict(), os.path.join(save_dir, "best_model.pt"))
        print("🎯 Best model updated.\n")
        min_val_loss = avg_val_loss
        early_stopping = 0
    else:
        early_stopping += 1
        if early_stopping >= patience:
            print("⛔ Early stopping devrede!")
            break

    scheduler.step()


Epoch 1 - Training:   0%|          | 0/3205 [00:00<?, ?it/s]<ipython-input-18-7c08244de16b>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validating:   0%|          | 0/357 [00:00<?, ?it/s]<ipython-input-18-7c08244de16b>:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validating: 100%|██████████| 357/357 [01:44<00:00,  3.40it/s]



Epoch 1: Train Loss = 5.1306 | Val Loss = 1.4920
✅ Model saved to: /content/drive/MyDrive/blip_checkpoints/epoch_01_val1.4920.pt
🎯 Best model updated.


Validating: 100%|██████████| 357/357 [01:44<00:00,  3.41it/s]



Epoch 2: Train Loss = 0.5392 | Val Loss = 0.2140
✅ Model saved to: /content/drive/MyDrive/blip_checkpoints/epoch_02_val0.2140.pt
🎯 Best model updated.


Validating: 100%|██████████| 357/357 [01:45<00:00,  3.38it/s]


...


Epoch 8: Train Loss = 0.0894 | Val Loss = 0.1022
✅ Model saved to: /content/drive/MyDrive/blip_checkpoints/epoch_08_val0.1022.pt
🎯 Best model updated.


Validating: 100%|██████████| 357/357 [01:45<00:00,  3.39it/s]



Epoch 9: Train Loss = 0.0867 | Val Loss = 0.1012
✅ Model saved to: /content/drive/MyDrive/blip_checkpoints/epoch_09_val0.1012.pt
🎯 Best model updated.


Validating: 100%|██████████| 357/357 [01:46<00:00,  3.36it/s]



Epoch 10: Train Loss = 0.0845 | Val Loss = 0.1004
✅ Model saved to: /content/drive/MyDrive/blip_checkpoints/epoch_10_val0.1004.pt
🎯 Best model updated.


In [21]:
import pickle
with open(os.path.join(save_dir, "/content/training_log.pkl"), "wb") as f:
    pickle.dump(tracking_info, f)